In [1]:
from samples import (
    get_data_samples,
    StimulationType,
)

data_samples = get_data_samples()

DataSample(force=-3.957, voltage=0.012045, acceleration_1=-0.0053978, acceleration_2=0.037517, acceleration_3=0.058993, stimulation_type=<StimulationType.MULTISINE_FULL_FREQUENCY_GRID: 'multisine_full_frequency_grid'>)

In [2]:
from training_validation_sets import get_training_validation_and_testing_sets

sets = get_training_validation_and_testing_sets(data_samples, 15)
(
    X_train,
    y_train,
    X_val,
    y_val,
    X_test,
    y_test,
) = sets

In [3]:
X_train.shape, y_train.shape

((63144,), (63144, 3))

In [4]:
X_train[0]

array([[26.2296  , -0.054031],
       [27.2707  , -0.056345],
       [27.8936  , -0.058008],
       [28.336   , -0.059006],
       [28.4586  , -0.059363],
       [28.1931  , -0.05905 ],
       [27.7373  , -0.058065],
       [26.8361  , -0.056436],
       [25.6448  , -0.05417 ],
       [24.2047  , -0.051318],
       [22.3752  , -0.047858],
       [20.4283  , -0.043878],
       [18.2511  , -0.039404],
       [15.7982  , -0.034486],
       [13.2947  , -0.02918 ]])

In [5]:
y_train[0]

array([0.19258, 0.13044, 0.12067], dtype=object)

In [6]:
import keras
import tensorflow as tf
import matplotlib as plt

sets = get_training_validation_and_testing_sets(
    data_samples, 15, StimulationType.SINE_SWEEP
)
X_train, X_val, X_test, y_train, y_val, y_test = sets

# Utworzenie modelu
model = tf.keras.Sequential(
    [
        tf.keras.layers.LSTM(128, activation="relu", input_shape=(X_train[0].shape)),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(3),
    ]
)

# Kompilacja modelu
model.compile(optimizer="adam", loss="mse")

# Trenowanie modelu
history = model.fit(
    X_train, y_train, validation_data=(X_val, y_val), epochs=3000, verbose=0
)

# Ocena modelu na zestawie testowym
loss = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss}")

# Wykres straty podczas trenowania
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss (MSE)")
plt.legend()
plt.title("Training and Validation Loss")
plt.show()

# Predykcja na zestawie testowym
predictions = model.predict(X_test)

# Wykres porównujący dane testowe z predykcjami
plt.figure(figsize=(12, 6))
for i in range(3):
    plt.plot(y_test[:, i], label=f"True Value {i+1}")
    plt.plot(predictions[:, i], label=f"Predicted Value {i+1}")
plt.xlabel("Samples")
plt.ylabel("Acceleration")
plt.title("Comparison of True and Predicted Values")
plt.legend()
plt.show()

2024-05-30 14:27:44.664346: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-30 14:27:44.789409: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-30 14:27:44.790320: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 14:27:46.730410: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TypeError: get_training_validation_and_testing_sets() takes 2 positional arguments but 3 were given